# CAPSTONE PROJECT  
**SEMANTIC SIMILARITY USING N-GRAMS AND NEURAL N-GRAMS.**

by NGOC (MIKE) TRAN




# 1.  Data prep

We can get the data directly from the nltk packet. 

In [1]:
import nltk
nltk.download("brown")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [2]:
brown = nltk.corpus.brown.words()

# This allows us to walk through the corpus word by word
print(brown)

# This behaves just like any other list
print(len(brown))
print(brown[0])
print(brown[:10])

brown = [word.lower() for word in brown]
print(brown[:20])

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]
1161192
The
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of']
['the', 'fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of', "atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that']


Spliting the data into train, dev, and test set

In [3]:
# Divide the data into training, evaluation and testing
train_words = brown[:800001]
dev_words = brown[800001:1000001]
test_words = brown[1000001:]

Build a vocabulary for the words while merging the words with frequency less than 4 into the same unknown category.

We also indexed all the words as it is the norm input for the neural network.

In [4]:
UNK = "<UNK>"
def build_indices(tokens, k):
  vocab = []
  word2idx = {}
  count = {}
  for i in tokens:
    if not (i in vocab):
      vocab.append(i)
      count[i] = 1
    else:
      count[i]+=1
    
  for j in list(count):
    if (count[j] <= k):
      del(count[j])
     
  vocab = list(count)
  vocab.append(UNK)
  for n in vocab:
    word2idx[n] = vocab.index(n)
  
  # Fill in your implementation here
  
  return word2idx, vocab

word2idx, vocab = build_indices(train_words, 3)

In [5]:
#Indexing all the dataset
train_indices = [word2idx[word] if word in vocab else word2idx[UNK] for word in train_words]
dev_indices = [word2idx[word] if word in vocab else word2idx[UNK] for word in dev_words]
test_indices = [word2idx[word] if word in vocab else word2idx[UNK] for word in test_words]

Check the size of the vocabulary

In [6]:
print(len(vocab)

14507


## 2. Ngrams Model
First, we take the count of all the unigrams, bigrams and trigrams

In [7]:
unigrams = {}
bigrams = {}
trigrams = {}

for word in train_indices:
  t = tuple((word,))
  if not(word in unigrams):
    unigrams[word] = train_indices.count(word)
    
trigram_set = [train_indices[i:i+3] for i in range(len(train_indices)-2)]
for word in trigram_set:
  t = tuple(word)
  trigrams[t] =  trigrams.get(t, 0) + 1
      
bigram_set = [train_indices[i:i+2] for i in range(len(train_indices)-1)]
for word in bigram_set:
  t = tuple(word)
  bigrams[t] =  bigrams.get(t, 0) + 1


Implement the interpolated n-grams

In [8]:
import numpy as np

def get_interpolated_trigram(unigrams, bigrams, trigrams, weights):
  # {unigrams, bigrams, trigrams}grams are the counts of the relevant ngram
  # weights is size-3 tuple that represents the weights of those model respectively, 
  # in probabilities that sum up to 1
  
  def interpolated_trigram(trigram):
    #Calculating the biggest grams first and goes down one by one
    #Minus to remove the redundant count since a bigger gram also contains the smaller one
    if (trigrams.get(trigram,0) == 0 and bigrams.get((trigram[0],trigram[1]),0) == 0):
      t3 =np.log(0)
    else:
      t3 = np.log(weights[2]) + np.log(trigrams.get(trigram,0)) - np.log(bigrams.get((trigram[0],trigram[1]),0))
    t2 = np.log(weights[1]) + np.log(bigrams.get((trigram[1],trigram[2]),0)) - np.log(unigrams.get(trigram[1],0))
    t1= np.log(weights[0]) + np.log(unigrams.get(trigram[2],0)) - np.log(len(unigrams))
    prob = np.logaddexp(t1, t2)
    prob2 = np.logaddexp(prob,t3)
    log_prob = prob2

    return log_prob
  
  return interpolated_trigram

interp_trigram = get_interpolated_trigram(unigrams, bigrams, 
                                          trigrams, (0.1, 0.3, 0.6))
#A quick test to see if our interpolated function is working
interp_trigram((1, 2, 3))

-2.2781149824005364

Calculating the perplexity

In [9]:
def perplexity(model, indices):
  perp = None
  
  perplexity = 0 #np.log(1)
  for i in range(len(indices)-2):
    prob = model((indices[i],indices[i+1],indices[i+2]))
    perplexity = perplexity + (1-prob)
  N = len(indices)-2
  perp = 1/N * perplexity
  
  return perp
  
  
perplexity(interp_trigram, dev_indices)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


4.9140115663064945

# 3. Neural Ngrams Model

We want to use the GPU support here

In [10]:
import torch
from torch import nn

# randomize the seed
torch.manual_seed(360)

if torch.cuda.is_available():
  torch.cuda.manual_seed_all(360)

Our neural trigram class with an overrided forward function

In [11]:
class NeuralTrigram(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size):
    #Call the constructor from the parent class
    super(NeuralTrigram, self).__init__()
    
    # The embedding layer
    self.embed = nn.Embedding(vocab_size, embed_size) 
    
    # Linearly transform to get to the hidden layer
    self.linear = nn.Linear(2 * embed_size, hidden_size)
    
    # Change the dimension to fit the hidden layer
    self.out = nn.Linear(hidden_size, vocab_size)
    
    # Apply a softmax to the first dimension of the hidden layer
    self.softmax = nn.LogSoftmax(dim=1)
    
    
  # Customizing the forward pass
  def forward(self, input):
    # Get the batch size and number of words
    batch_size, n_words = input.shape

    # embedding the input
    embeds = self.embed(input)
    
    # Reshape the vector size to fit the context
    context = embeds.view((batch_size, -1))
    
    # Put the context through a tanh function
    hidden = torch.tanh(self.linear(context))
    
    # Get a score for the next word and softmax it so it will fit into a probability distribution
    scores = self.out(hidden)
    probs = self.softmax(scores)
    
    return probs

Our evaluation code

In [12]:
def evaluate(model, indices, batch_size=32, cuda=False):
  model.eval()
  with torch.no_grad():
    if cuda:
      model = model.cuda()
    total_loss = 0.0
    
    # Iterating over the words, stepping over batch_size each time
    for i in range(2, len(indices)-batch_size+1, batch_size):
      # build a tensor for the batch [i, i+batch_size]
      input = torch.tensor([indices[j-2:j] for j in range(i, i + batch_size)], 
                           dtype=torch.int64)
      if cuda:
        input = input.cuda()
      target = torch.tensor(indices[i:i+batch_size], dtype=torch.int64)

      model.zero_grad()
      output = model.forward(input)
      if cuda:
        output = output.cpu()
        
      # Compute the loss, we use the negative log likelihood loss
      loss = nn.NLLLoss()(output, target)
      total_loss += loss.item()
      
    # Averaging out the loss
    num_batches = (len(indices)-2)//batch_size
    avg_loss = total_loss/num_batches

    return avg_loss

Training code:
For each batch of the training set, we do a forward pass, compute the respective score, and then do a backward pass by calling the backward function on the loss, and telling the optimizer to step backward to update the parameters

In [13]:
import time
def train(model, indices, dev_indices, batch_size=32, n_epochs=1, cuda=False):
  
  #We use Stochastic Gradient Descent as our optimizer
  optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, weight_decay=0.00001)
  if cuda:
    model = model.cuda()
  
  losses = []
  
  for epoch in range(n_epochs):
    start_time = time.time()
    model.train()
    
    total_loss = 0.0
    for i in range(2, len(indices)-batch_size+1, batch_size):
      #Similar to evaluation code
      input = torch.tensor([indices[j-2:j] for j in range(i, i + batch_size)], 
                           dtype=torch.int64)
      target = torch.tensor(indices[i:i+batch_size], dtype=torch.int64)

      if cuda:
        input = input.cuda()
        
      optimizer.zero_grad()

      #Compute the loss
      log_prob = model(input)
      if cuda:
        log_prob = log_prob.cpu()
      loss = nn.NLLLoss()(log_prob, target)
      
      #Backward pass to update the gradients and the parameters
      loss.backward()
      total_loss += loss.item()
      optimizer.step()
      
    num_batches = (len(indices)-2)//batch_size
    avg_loss = total_loss/num_batches
    losses.append(avg_loss)
    
    # check the current loss for the dev set
    dev_loss = evaluate(model, dev_indices, cuda=cuda)
    
    #Calculate perplexity
    perp = pow(np.e,avg_loss)
      
    #Logging
    print("epoch {} | time {:.2} | train loss {:.5} | dev loss {:.5} | perp {:4.2}".format(
            epoch + 1, time.time() - start_time, avg_loss, dev_loss, perp))
  return losses

Let's train our model!


In [44]:
print("----Beginning Training----")
ntrigram = NeuralTrigram(vocab_size=len(vocab), embed_size=30, hidden_size=50)
train(ntrigram, train_indices,dev_indices, n_epochs=20, cuda=True)
#evaluate(ntrigram,train_indices)

----Beginning Training----
epoch 1 | time 7.2e+01 | train loss 6.186 | dev loss 5.8917 | perp 4.9e+02
epoch 2 | time 7.2e+01 | train loss 5.9076 | dev loss 5.7952 | perp 3.7e+02
epoch 3 | time 7.1e+01 | train loss 5.815 | dev loss 5.7465 | perp 3.4e+02
epoch 4 | time 7.1e+01 | train loss 5.7538 | dev loss 5.7148 | perp 3.2e+02
epoch 5 | time 7.2e+01 | train loss 5.707 | dev loss 5.6911 | perp 3e+02
epoch 6 | time 7.1e+01 | train loss 5.6682 | dev loss 5.6732 | perp 2.9e+02
epoch 7 | time 7.1e+01 | train loss 5.6348 | dev loss 5.6595 | perp 2.8e+02
epoch 8 | time 7.2e+01 | train loss 5.6052 | dev loss 5.6478 | perp 2.7e+02
epoch 9 | time 7.2e+01 | train loss 5.5785 | dev loss 5.6376 | perp 2.6e+02
epoch 10 | time 7.1e+01 | train loss 5.5542 | dev loss 5.6288 | perp 2.6e+02
epoch 11 | time 7.2e+01 | train loss 5.5319 | dev loss 5.6227 | perp 2.5e+02
epoch 12 | time 7.2e+01 | train loss 5.511 | dev loss 5.6174 | perp 2.5e+02
epoch 13 | time 7.3e+01 | train loss 5.4916 | dev loss 5.6115 | 

[6.186001565619571,
 5.9075649293156935,
 5.815048782390291,
 5.753797667012882,
 5.707020943172169,
 5.668243611563234,
 5.634787911104914,
 5.605184354974373,
 5.578459259500369,
 5.5542363057094,
 5.53187491836641,
 5.511032267119237,
 5.491590454652885,
 5.47327030313993,
 5.455901750350333,
 5.439483412332862,
 5.424012527660759,
 5.409217341059976,
 5.395075521314996,
 5.381413503791662]

Testing both models on the test set

In [45]:
evaluate(ntrigram,test_indices,cuda=True)


5.590502789504759

In [27]:
perplexity(interp_trigram,test_indices)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


4.87501882462981

# 4. Semantic Similarity
Let's just define the cosine similarity function

In [17]:
def cosine_sim(a, b):
  similarity = 0
  similarity = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))  
  return similarity

Explore the model with some interesting pair of words

In [30]:
# This long sequence of calls gets us a matrix representing the embeddings we trained
embed_matrix = ntrigram.cpu().embed.weight.data.numpy()

a = embed_matrix[word2idx["car"]]
b = embed_matrix[word2idx["dog"]]
c = embed_matrix[word2idx["cat"]]

cosine_sim(a, b), cosine_sim(b, c)

(-0.14776057, -0.009813079)

A classic example 

In [32]:
spear, sword, rock = "spear", "sword", "rock"
a = embed_matrix[word2idx[spear]]
b = embed_matrix[word2idx[sword]]
c = embed_matrix[word2idx[rock]]

cosine_sim(a, b), cosine_sim(a, c)

(-0.18928087, -0.22034152)

In [37]:
atlanta, city, place = "atlanta", "city", "place"
a = embed_matrix[word2idx[atlanta]]
b = embed_matrix[word2idx[city]]
c = embed_matrix[word2idx[place]]

cosine_sim(a, b), cosine_sim(a, c)

(-0.086587526, -0.033710107)

In [38]:
cosine_sim(embed_matrix[word2idx['inadequate']],embed_matrix[word2idx['ambiguous']]), cosine_sim(embed_matrix[word2idx['inadequate']],embed_matrix[word2idx['experienced']])

(-0.23602815, -0.1257164)

In [39]:
cosine_sim(embed_matrix[word2idx['husband']],embed_matrix[word2idx['son']]), cosine_sim(embed_matrix[word2idx['wife']],embed_matrix[word2idx['daughter']])

(0.5580224, 0.06786769)